In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 5.6MB/s 
     |████████████████████████████████| 3.0MB 26.7MB/s 
     |████████████████████████████████| 1.1MB 56.5MB/s 
     |████████████████████████████████| 890kB 51.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=11cf182d809127037b7a5d7bfac37089967f7a3a126b2535733f1498efe649e9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import io
import torch
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from keras.preprocessing.sequence import pad_sequences
from transformers import AdamW, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Found GPU at: /device:GPU:0


'Tesla T4'

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive/
/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('My Drive/Colab_Notebooks/selected_electronics.csv')
df.head()

,sentence,label
0,i hate the thing that in the ad they say micro...,1
1,very good product in this price range,4
2,great works nicely functions are smooth howeve...,4
3,not good enough for gaming i installed blur ga...,2
4,design and performance wise good gtx 1650 is w...,3


In [ ]:
df.label = df.label-1
np.unique(df.label)

array([0, 1, 2, 3, 4])

In [ ]:
max_length = 40
learning_rate = 2e-5
batch_size = 64
epochs = 4

In [ ]:
# Create sentence and label lists
initial_sentences = df.sentence.values
initial_labels = df.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [ tokenizer.tokenize(sent) for sent in initial_sentences]
# print ("tokenized_texts:")
# print (tokenized_texts[0])

index = []
for i,x in enumerate(tokenized_texts):
    if len(x)<max_length:
        index.append(i)
sentences = []
labels=[]
for i in index:
   sentences.append(initial_sentences[i]) 
   labels.append(initial_labels[i])


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [["[CLS]"] + tokenizer.tokenize(sent)  + ["[SEP]"] for sent in sentences]
# print ("tokenized_texts:")
# print (tokenized_texts[0])


input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad input tokens
input_ids = pad_sequences(input_ids, maxlen=max_length, dtype="long", truncating="post", padding="post")

# make attention masks
attention_masks = []
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)


train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=1000, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=1000, test_size=0.1)

# Convert all of our data into torch tensors
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Create an iterator
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)
model.cuda()

parameter_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in parameter_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in parameter_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=learning_rate,
                     )

def flatten_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# Store our loss and accuracy for plotting
train_loss_set = []

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  # Training
  
  model.train()
  training_loss = 0
  nbatch_training_examples, nb_training_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    batch_input_ids, batch_input_mask, batch_labels = batch
    optimizer.zero_grad()
    # Forward pass
    loss = model(batch_input_ids,  attention_mask=batch_input_mask, labels=batch_labels)
    train_loss_set.append(loss[0])    
    # # Backward pass
    loss[0].backward()
    optimizer.step()
    
    
    # Update tracking variables
    training_loss =training_loss+ loss[0]
    nbatch_training_examples =nbatch_training_examples+ batch_input_ids.size(0)
    nb_training_steps =nb_training_steps+ 1

  print("Train loss: {}".format(training_loss/nb_training_steps))
    
    
  # Validation
  model.eval()

  evaluation_loss, evaluation_accuracy = 0, 0
  nbatch_evaluation_steps, nbatch_evaluation_examples = 0, 0
  eval_f1, eval_precision = 0,0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    batch_input_ids, batch_input_mask, batch_labels = batch
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(batch_input_ids,  attention_mask=batch_input_mask)[0]
     
    logits = logits.detach().cpu().numpy()
    label_ids = batch_labels.to('cpu').numpy()

    tmp_evaluation_accuracy = accuracy_score(np.argmax(logits, axis=1).flatten(), label_ids.flatten()) 
    eval_f1 += f1_score(np.argmax(logits, axis=1).flatten(), label_ids.flatten(), average='weighted') 
    eval_precision += precision_score(np.argmax(logits, axis=1).flatten(), label_ids.flatten(), average='weighted')

    evaluation_accuracy = evaluation_accuracy+tmp_evaluation_accuracy
    nbatch_evaluation_steps =nbatch_evaluation_steps+ 1

  print("Validation Accuracy: {}".format(evaluation_accuracy/nbatch_evaluation_steps))
  print("Validation F1 score: {}".format(eval_f1/nbatch_evaluation_steps))
  print("Validation precision: {}".format(eval_precision/nbatch_evaluation_steps))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.9957111477851868


Epoch:  25%|██▌       | 1/4 [03:54<11:43, 234.54s/it]

Validation Accuracy: 0.7748742816091955
Validation F1 score: 0.7893340917048334
Validation precision: 0.8303281116072615
Train loss: 0.4545663893222809


Epoch:  50%|█████     | 2/4 [07:52<07:50, 235.47s/it]

Validation Accuracy: 0.8562679597701149
Validation F1 score: 0.860644391935123
Validation precision: 0.8771188694695922
Train loss: 0.2806352972984314


Epoch:  75%|███████▌  | 3/4 [11:50<03:56, 236.20s/it]

Validation Accuracy: 0.896210488505747
Validation F1 score: 0.8973108003690992
Validation precision: 0.9039760863458649
Train loss: 0.18355892598628998


Epoch: 100%|██████████| 4/4 [15:48<00:00, 237.01s/it]

Validation Accuracy: 0.9201867816091953
Validation F1 score: 0.9212980361471623
Validation precision: 0.9295339094187027
